In [125]:
import sqlite3

In [126]:
# open a sqlite database
conn = sqlite3.connect('darkwebreports.db')

In [127]:
tablename = 'pings'

In [128]:
# define columns as a list of pairs with title and type
columns = [
    ('HiddenService', 'TEXT'),
    ('Language', 'short_string CHAR(2)'),
    ('HTTPStatus', 'INTEGER'),
    ('FirstSeen', 'TEXT'),
    ('LastContacted', 'TEXT'),
    ('LastUp', 'TEXT'),
    ('Title', 'TEXT'),
    ('Sheetname', 'TEXT'),
]
# TODO: combine date columns First Seen and LastContacted into one date column
# TODO: store dateime as int (and/or split date/time) to put an index on the date
# TODO: consider more indexes          

In [129]:
def generateCreateTableQuery(tablename, columns):
    query = 'CREATE TABLE "' + tablename + '" ('
    for column in columns:
        query += '\n"' + column[0] + '" ' + column[1] + ','
    query = query[:-1] + ');'
    print(query)
    return query

In [130]:
conn.execute(
    generateCreateTableQuery(tablename, columns)
)

CREATE TABLE "pings" (
"HiddenService" TEXT,
"Language" short_string CHAR(2),
"HTTPStatus" INTEGER,
"FirstSeen" TEXT,
"LastContacted" TEXT,
"LastUp" TEXT,
"Title" TEXT,
"Sheetname" TEXT);


In [131]:
# create an index on the column "Hidden Service"
conn.execute('''CREATE INDEX "''' + tablename + '''_HiddenService" ON ''' + tablename + '''("HiddenService");''')        

In [132]:
import pandas as pd
import os

# iterate through the .xlsx files in the rawreports folder and add it to a list of files
files = []
for file in os.listdir('rawreports'):
    if file.endswith('.xlsx'):
        files.append(file)


In [133]:
#only take the first files for testing purposes
files = files[:1]
len(files)

1

In [134]:
sheets = ["New Today", "Down", "Up"]

In [135]:
def import_sheet(sheetname, file):
    # read the excel file into a pandas dataframe
    df = pd.read_excel('rawreports/' + file,sheetname)
    
    df.columns = df.columns.map(lambda x: x.replace(' ', ''))
    # add a column for the sheetname
    df['Sheetname'] = sheetname
    # convert the dataframe to a sqlite table
    df.to_sql(tablename, conn, if_exists='append', index=False)
    return df

In [136]:
# define df as empty df to store the result for debugging purposes in the notebook
df = pd.DataFrame()

for file in files:
    for sheetname in sheets:
        print(file, sheetname)
        df = import_sheet(sheetname, file)
        print(len(df))
#print(df)

HiddenServices-2018-2-14.xlsx New Today
3
HiddenServices-2018-2-14.xlsx Down
11735
HiddenServices-2018-2-14.xlsx Up
4838


In [137]:
#df

In [138]:
conn.execute('VACUUM')
conn.commit()
conn.close()